In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 이미지 처리
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
# json 파일 다루기
import json

In [ ]:
# 파일 존재 여부 확인용
import os

In [ ]:
# 훈련용 데이터와 검증용 데이터 분리
from sklearn.model_selection import train_test_split

# 디렉토리 설정

In [ ]:
os.chdir('/content/drive/MyDrive/[perst]데이터셋/MachineLearningProject/01_Cloth Detection Model')

In [ ]:
os.getcwd()

# 변수 목록

In [ ]:
# 사용할 이미지 개수
img_file_size = 2000

# 크기 조정 사이즈
res_width = 200
res_height = 200

ori_img_dir_name = "./Dataset/Original/Image/" # 원본 이미지 파일이 존재하는 디릭토리명
ori_json_dir_name = "./Dataset/Original/Json/" # 원본 json 파일이 존재하는 디렉토리명

train_dir_name = "./Dataset/Train-Test_All/Train/" # Train 데이터를 저장할 디렉토리명
test_dir_name = "./Dataset/Train-Test_All/Test/" # Test 데이터를 저장할 디렉토리명

# 반복실행용 변수

In [ ]:
# end_read_point = 0
# end_save_point_train = 0
# end_save_point_test = 0

In [ ]:
end_read_point = 8000
end_save_point_train = 6400
end_save_point_test = 1600

In [ ]:
start_read_point = end_read_point + 1
start_save_point_train = end_save_point_train + 1
start_save_point_test = end_save_point_test + 1

In [ ]:
print('End Read Point : ' + str(end_read_point))
print('Start Read Point : ' + str(start_read_point))
print()
print('End Save Point (Train) : ' + str(end_save_point_train))
print('Start Save Point (Train) : ' + str(start_save_point_train))
print()
print('End Save Point (Test) : ' + str(end_save_point_test))
print('Start Save Point (Test) : ' + str(start_save_point_test))

# 데이터 불러오기

In [ ]:
dataset_list = []
read_point = start_read_point - 1

while (len(dataset_list) < img_file_size):
  read_point = read_point + 1

  img_path = ori_img_dir_name + str(read_point) + ".jpg"
  json_path = ori_json_dir_name + str(read_point) + ".json"

  # 해당 숫자의 파일이 없으면 다음 숫자 뽑기
  if not os.path.isfile(img_path):
    continue
  if not os.path.isfile(json_path):
    continue

  tmp_list = []

  # 이미지 파일 로드
  img_file = cv2.imread(img_path)
  if img_file is None:
    print(str(read_point) + ".jpg File Load Failed!!")
    break

  # json 파일 로드
  with open(json_path, "r") as loadfile:
    json_file = json.load(loadfile)
    if json_file is None:
      print(str(read_point) + ".json File Load Failed!!")
      break

  tmp_list.append(img_file)
  tmp_list.append(json_file)

  dataset_list.append(tmp_list)
  end_read_point = read_point

  if ((len(dataset_list))%500) == 0: # 진행도 체크용
    print(str(len(dataset_list)) + "개 파일 읽어오기 완료")

print()
print("Complete")
print("End Read Point : " + str(end_read_point))

In [ ]:
len(dataset_list)

In [ ]:
len(dataset_list[0])

# 폴리곤 좌표에서 라벨링 마스크 생성

## 함수

In [ ]:
# 폴리곤 좌표로 라벨링 마스크를 그리는 함수
# 입력 형태: json 파일 통째로
# 출력 형태: 라벨링된 마스크 이미지

def poly_to_mask(json_dict):
    # 빈 마스크 생성
    img_mask = np.zeros((json_dict['Img_Height'], json_dict['Img_Width']))

    # 세그멘테이션 영역 이름 리스트
    part_list = list(json_dict['Poly_Seg'].keys())

    for part in part_list:
        if part == 'Outer':
            mask_pts = np.array(json_dict['Poly_Seg'][part], np.int32)
            cv2.fillPoly(img_mask, [mask_pts], 1)
        if part == 'Top':
            mask_pts = np.array(json_dict['Poly_Seg'][part], np.int32)
            cv2.fillPoly(img_mask, [mask_pts], 2)
        if part == 'Bottom':
            mask_pts = np.array(json_dict['Poly_Seg'][part], np.int32)
            cv2.fillPoly(img_mask, [mask_pts], 3)
        if part == 'Onepiece':
            mask_pts = np.array(json_dict['Poly_Seg'][part], np.int32)
            cv2.fillPoly(img_mask, [mask_pts], 4)

    # Float 형식을 Int 형식으로 변경
    img_mask = img_mask.astype('uint8')

    return img_mask


## 적용

In [ ]:
# dataset_list[i][2]에 세그멘테이션 이미지를 저장
# dataset_list[i][0]은 원본 이미지, [1]은 json, [2]는 마스크 이미지

for data in dataset_list:
    data.append(poly_to_mask(data[1]))

print("Complete")

In [ ]:
# 리스트 형태 확인
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(dataset_list[0][0])
plt.title("Original Image")

plt.subplot(1, 2, 2)
plt.imshow(dataset_list[0][2])
plt.clim(0, 4)
plt.colorbar(shrink=0.5)
plt.title("Labeling")

plt.show()

# 이미지 크기 조정

In [ ]:
res_img_sample = cv2.resize(dataset_list[0][0], dsize=(res_height, res_width), interpolation=cv2.INTER_AREA)
res_mask_sample = cv2.resize(dataset_list[0][2], dsize=(res_height, res_width), interpolation=cv2.INTER_NEAREST)

In [ ]:
np.unique(res_mask_sample)

In [ ]:
# 리스트 형태 확인
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(res_img_sample)
plt.title("Original Image")

plt.subplot(1, 2, 2)
plt.imshow(res_mask_sample)
plt.clim(0, 4)
plt.colorbar(shrink=0.5)
plt.title("Labeling")

plt.show()

## 전체 이미지 크기 조정

In [ ]:
for data in dataset_list:
    data[0] = cv2.resize(data[0], dsize=(res_height, res_width), interpolation=cv2.INTER_AREA)
    data[2] = cv2.resize(data[2], dsize=(res_height, res_width), interpolation=cv2.INTER_NEAREST)

print("Complete")

# 변경된 이미지에 맞추어 Json 파일값 조정

## 이미지 Width, Height 정보 수정 및 렉트 좌표값 정보 삭제

In [ ]:
dataset_list[0][1].keys()

In [ ]:
for data in dataset_list:
    data[1]['Img_Width'] = res_width
    data[1]['Img_Height'] = res_height
    del data[1]['Rect_Seg']

print("Complete")

In [ ]:
dataset_list[0][1]

# Train-Test Split

In [ ]:
# 8:2 비율로 train-test split
train, test = train_test_split(dataset_list, test_size=0.2) # stratify, random_state 설정 가능

In [ ]:
len(train)

In [ ]:
len(test)

# 좌표 정보 통계

## Train 데이터 좌표 정보 통계

In [ ]:
seg_check_list = []

for i in range(0, len(train)):
    seg_name_list = list(train[i][1]['Poly_Seg'].keys())

    seg_check_list.append(seg_name_list)

    if ((i+1)%100) == 0: # 진행도 체크용
        print(str(i+1) + "개 정보 읽어오기 완료")


seg_count = {}

for item in seg_check_list:
    if str(item) not in seg_count: # 해당 키가 없을 경우
        seg_count[str(item)] = 1
    else:
        seg_count[str(item)] = seg_count[str(item)]+1


seg_count = sorted(seg_count.items(), key= lambda item:item[1], reverse=True)

seg_count_labels = []
seg_count_values = []

for item in seg_count:
    seg_count_labels.append(item[0])
    seg_count_values.append(item[1])


print("Complete")

In [ ]:
barplot = plt.bar(seg_count_labels, seg_count_values)
plt.xticks(rotation=90)

for i, j in enumerate(barplot) :
    plt.text(i, j.get_height() + 0.5, seg_count_values[i], ha = 'center')

plt.title("Train Data Segmentation Count")
plt.show()

In [ ]:
total = np.sum(seg_count_values)
seg_count_labels_ratio = []

for i in range(0, len(seg_count_labels)):
    ratio = (seg_count_values[i]/total) * 100

    new_label = seg_count_labels[i] + (' %.1f%%' % ratio)
    seg_count_labels_ratio.append(new_label)

print("Complete")

In [ ]:
plt.pie(seg_count_values, startangle=90, counterclock=False)

plt.title("Train Data Segmentation Count")
plt.legend(seg_count_labels_ratio, loc=(1.0, 0.3))
plt.show()

## Test 데이터 좌표 정보 통계

In [ ]:
seg_check_list = []

for i in range(0, len(test)):
    seg_name_list = list(test[i][1]['Poly_Seg'].keys())

    seg_check_list.append(seg_name_list)

    if ((i+1)%100) == 0: # 진행도 체크용
        print(str(i+1) + "개 정보 읽어오기 완료")


seg_count = {}

for item in seg_check_list:
    if str(item) not in seg_count: # 해당 키가 없을 경우
        seg_count[str(item)] = 1
    else:
        seg_count[str(item)] = seg_count[str(item)]+1


seg_count = sorted(seg_count.items(), key= lambda item:item[1], reverse=True)

seg_count_labels = []
seg_count_values = []

for item in seg_count:
    seg_count_labels.append(item[0])
    seg_count_values.append(item[1])


print("Complete")

In [ ]:
barplot = plt.bar(seg_count_labels, seg_count_values)
plt.xticks(rotation=90)

for i, j in enumerate(barplot) :
    plt.text(i, j.get_height() + 0.5, seg_count_values[i], ha = 'center')

plt.title("Test Data Segmentation Count")
plt.show()

In [ ]:
total = np.sum(seg_count_values)
seg_count_labels_ratio = []

for i in range(0, len(seg_count_labels)):
    ratio = (seg_count_values[i]/total) * 100

    new_label = seg_count_labels[i] + (' %.1f%%' % ratio)
    seg_count_labels_ratio.append(new_label)

print("Complete")

In [ ]:
plt.pie(seg_count_values, startangle=90, counterclock=False)

plt.title("Test Data Segmentation Count")
plt.legend(seg_count_labels_ratio, loc=(1.0, 0.3))
plt.show()

# 학습용 데이터를 X와 Y로 구분

In [ ]:
x_train = []
json_train = []
poly_train = []

for data in train:
    x_train.append(data[0])
    json_train.append(data[1])
    poly_train.append(data[2])

print("Complete")

In [ ]:
x_test = []
json_test = []
poly_test = []

for data in test:
    x_test.append(data[0])
    json_test.append(data[1])
    poly_test.append(data[2])

print("Complete")

# 학습용 데이터를 폴더에 저장

In [ ]:
palette = [0,0,0,
          255,0,0,
          0,255,0,
          0,0,255,
          255,255,0]

In [ ]:
i = start_save_point_train

for img in x_train:
    success = cv2.imwrite(train_dir_name + "Image/" + str(i) + ".jpg", img)

    if not success:
        print("Image Save Failed!!")
        break

    i = i+1

print("Complete")

In [ ]:
i = start_save_point_train

for mask in poly_train:
    png_mask = Image.fromarray(mask).convert('P')
    png_mask.putpalette(palette)

    mask_path = train_dir_name + "Annotation/" + str(i) + ".png"
    png_mask.save(mask_path)

    if not os.path.isfile(mask_path):
        print("Image Save Failed!!")
        break

    i = i+1

print("Complete")

In [ ]:
i = start_save_point_train

for json_file in json_train:
    json_path = train_dir_name + "Json/" + str(i) + ".json"

    with open(json_path, 'w') as outfile:
        json.dump(json_file, outfile)

    if not os.path.isfile(json_path): # 파일 저장에 실패했을 경우
        print("Json Save Failed!!")
        break

    end_save_point_train = i
    i = i+1

print("Complete")

In [ ]:
i = start_save_point_test

for img in x_test:
    success = cv2.imwrite(test_dir_name + "Image/" + str(i) + ".jpg", img)

    if not success:
        print("Image Save Failed!!")
        break

    i = i+1

print("Complete")

In [ ]:
i = start_save_point_test

for mask in poly_test:
    png_mask = Image.fromarray(mask).convert('P')
    png_mask.putpalette(palette)

    mask_path = test_dir_name + "Annotation/" + str(i) + ".png"
    png_mask.save(mask_path)

    if not os.path.isfile(mask_path):
        print("Image Save Failed!!")
        break

    i = i+1

print("Complete")

In [ ]:
i = start_save_point_test

for json_file in json_test:
    json_path = test_dir_name + "Json/" + str(i) + ".json"

    with open(json_path, 'w') as outfile:
        json.dump(json_file, outfile)

    if not os.path.isfile(json_path): # 파일 저장에 실패했을 경우
        print("Json Save Failed!!")
        break

    end_save_point_test = i
    i = i+1

print("Complete")

# 실제 폴더 확인